# Final Project Report:
## Galactic Distance Estimates from Photographs
#### Author: Julieanna Bacon

[Github Link!](https://github.com/JuLieAlgebra/cv_final_project)


# Table of contents
1. [Definitions](#def)
1. [Problem Background](#introduction)
2. [Project Overview](#paragraph1)
    1. [Acquiring the Data](#gettingdata)
        1. [Code](#download_code)
    2. [Preprocessing](#preprocessing)
        2. [Code](#preprocessing_code)
3. [EDA](#eda)
    1. [Preprocessed Data: Visualized](#eda_vis)
    2. [Stats: Visualized](#eda_stats)
4. [Architecture](#architecture)
    1. [Code](#arch_code)
    2. [Training Ex](#train_code)
5. [Improving the Model](#improve)
6. [Results](#results)
7. [Sources](#sources)

### Definitions <a name="def"></a>
- Redshift – “distance” as measured by Doppler shift. 
- Hubble Constant – a time varying parameter that tells us the rate of expansion of the universe at that time epoch 
- CCD – the camera of choice for most telescopes 
- Spectroscopic Redshift – more accurate than photometric redshifts. Distance measurements produced by fitting spectra data to a black body curve 
- Photometric Redshift - distance measurements produced by photometric data 
- Photometric – Data produced by photos of objects. Usually taken in a filter to only capture photos in a given range of wavelengths (red, blue, green, etc). 
- SDSS – Sloan Digital Sky Survey

## Problem Background <a name="introduction"></a>
I loosely replicated (and modified) the smaller architecture from this paper: Photometric redshifts from SDSS images using a Convolutional Neural Network [2]. 

The overall goal of the project is to produce distance estimations of galaxies from photos in a learned approach. Below is a quick overview of the science behind why this is possible.

How far away is this galaxy?

<img src='docs/images/galaxy_dist.png' width="400" height="400">

In astronomy, until we develop light speed engines, we’re almost entirely limited to estimating this by only cameras. It’s a surprisingly hard task that’s far from solved. It also underpins every aspect of research. From determining the age of the universe and resolving crises in fundamental physics to being able to calibrate solar convection models, it impacts everything.

Astronomers have developed a variety of methods, used for different distances and different situations, but we will talk about the most accurate and its learned approximation.

<img src='docs/images/slide1.png' width="700" height="500">  

This method is called spectroscopic redshift. Spectroscopic, meaning it looks at the spectrum, and redshift, the distance to the galaxy. We’re all familiar with Doppler Shift, as we experience it every time an ambulance passes us on the street. It describes how waves, sound or light, get stretched out or compressed as objects move relative to each other. And galaxies are, unfortunately and fortunately, all moving away from us. This is because the universe itself is expanding, and like ants on a balloon being blown up, ants/galaxies farther away from us appear to moving faster away than nearby ants.

<img src='docs/images/cosmic_doppler.png' width="400" height="400">  
<!-- <img src='docs/images/cosmological_redshift.png' width="400" height="400"> 
 -->
With expansion, the light from galaxies is being stretched out, and the farther away from us, the more their light has been stretched out. This stretching out of light makes the whole object seem slightly redder (thus the term redshift). We can measure how the light has become stretched out, and thus how far away it is, through spectroscopy very precisely, as spectra shows us how bright the object is at every wavelength. If we know that the object will be super bright at wavelength 4268nm (for example) because it’s mostly made of hydrogen and hydrogen glows very brightly at that wavelength, we can see how much that hydrogen line has shifted when we observe the object.
<p float="left">
  <img src='docs/images/spectra_elements.png' width="400" />
  <img src='docs/images/ex_spectrum.png' width="400" /> 
</p>

Unfortunately, getting spectra is a hard process that’s not always possible.

<img src='docs/images/slide2.png' width="700" height="400">  

The alternative is to see how bright the image via photos, which you can think of as a rough histogram of a spectrum. Spectra show brightness at every wavelength, photographs show brightness over a range of wavelengths. Astronomers use filters, like green, ultraviolet, etc to take photos in narrower wavelength bands than our phone cameras to see how bright something is over that histogram bin. Since there’s still information about how the object has been “reddened” in that rough histogram, astronomers can still get a (rougher) estimate of that reddening. This process is called Photometric Redshift.

While with images instead of spectra, you won't be able to match up the the distinct emission lines from elements to "aha!" precisely how much the object has been reddened, there are still real, physically meaningful relationships that these machine learning methods can learn from this photometric data. A few of them are also fleshed out distance methods of their own: https://en.wikipedia.org/wiki/Cosmic_distance_ladder if you'd like to read more.

<img src='docs/images/problem_overview.png' width="700" height="400">   

For photometric redshifts, most astronomy pipelines today transform a cleaned image of an object in a particular filter to a single number, magnitude/brightness, that can then be used in a machine learning approach. There are other things calculated from these images that are used in other situations, but for redshift estimation, the work is done with usually five or more magnitudes in different, non-overlapping filters.

I think this quote from a recent photometric redshift with CNNs paper sums up the promise of using the *entire* image instead of just the singular numbers:  

"A challenge for photo-z [photometric redshift] estimation methods that take magnitudes and colours as inputs is that there is not enough information...to break various degeneracies in the colour–redshift relation. One way to break these degeneracies is to include information about morphology, orientation, surface brightness, or visual appearance in general... A galaxy may appear red not just because its stellar population is instrinsically red but because it is a dusty edge-on spiral galaxy. Moreover, the fact that farther objects appear to be smaller and fainter to an observer also give us an additional piece of information to help break degeneracies. Most traditional methods for quantifying galaxy morphology, like ellipticity and Sérsic index, cannot fully encode all of the visual information that an image of a galaxy provides and hence methods that use images of galaxies directly as inputs ... and rely on artificial neural networks are the current state-of-the-art."

## Project Overview <a name="paragraph1"></a>

The goal of this project is to produce distance estimates for galaxies via images of them in different filter bands. Below is an illustration of the filter bands used by the SDSS, which is where we obtained all of our data.
<img src='docs/images/sdss_filters.png' width="400" height="400">

Originally I wanted to loosely replicate the Pasquet paper for a small dataset and play with different architectures, including a mixed-input model. I underestimated how long it would take to acquire and process all the data and produce a working model, so I did not get to the mixed-input modifications for the paper.

In Pasquet et al, their main architecture is for 500,000+ galaxies, with them mentioning a smaller architecture that did fairly well on 10,000 galaxies. Due to hardware and time constraints, I chose the smaller dataset size, which still totalled to 80GB of compressed raw data. 

However, they didn't actually specify their smaller architecture or details like loss functions, optimizers, or parameters in the paper itself. I found their code repo that was meant to replicate their state of the art architecture at the time, but it performly extremely poorly out of the box. I detail that in the Results & Improving the Model sections.

On the technical note, for most of the downloading and preprocessing, I used a framework called [Luigi](https://luigi.readthedocs.io/en/stable/). It allows you to actually parallelize your work and organize a set of steps (downloading, cleaning, cropping, etc) for Luigi to handle kicking off and ensuring that everything runs smoothing. It alerts you when something fails as well, you can also use it to run on servers, etc, it's very much like a Python-centric Airflow that's more hands on (and open source). 

With the 50,000+ images that I needed to download and process, it was pretty essential to my sanity.

## Acquiring the Data <a name="gettingdata"></a>

The tabular data and the image files are all available through the Sloan Digital Sky Survey (SDSS). They have a number of ways of acquiring both forms of data, though for bulk image data downloads, the recommendation is to use wget.

I needed to create a list of all the objects I wanted in this dataset and from the information in that list, grab all of the images in the five filter bands I was interested in for each object (which are galaxies in this case). This involved submitting an SQL query to the CasJobs site. I used a different data release than the paper I’m (loosely) replicating and had different criteria for redshift range, number of objects that I could train on, and need to randomize the order. However, the database schema stayed relatively similar between the original paper and the data release I used (DR17 instead of DR12), so most of the query remains very similar to the original publish query.
<img src='docs/images/sql.png' width="700" height="400">

Once I had the results of that, I downloaded the database as a CSV and pushed it to my own S3 bucket. With a secure endpoint for these calculations, to make sure everything was perfectly repeatable (I have set seeds for everything), I wrote a S3 target task for luigi and a downloader for the tabular data (the results of the query).
<img src='docs/images/tabular_data.png' width="900" height="400">

Above you can see a snippet of the tabular data. LibreOffice Calc isn’t doing a great job of parsing the file, but pandas did everything correctly. From the columns camcol, field, and rerun, you can determine which FITS image the observation is in and grab each observation from formatting like this: f”https://data.sdss.org/sas/dr17/eboss/photoObj/frames/{row.rerun}/{row.run}/{row.camcol}/frame-{band}-{str(row.run).zfill(6)}-{row.camcol}-{str(row.field).zfill(4)}.fits.bz2n”

Calling wget on that url will get you the right file. I generated a temporary file name based on the hash of the formatted url and told wget to download the file to that temp name, then when it was done, atomically “move” the file to its final name. I wrapped everything in a try except so it would continue if it ran into errors, since each downloading task is reponsible for downloading thousands of files.

I also check to make sure that if the file already exists, not to download it. If the downloading failed, and the temporary file got deleted, I also tell luigi to re-attempt downloading.

Also, unlike the Pasquet paper, I was getting galaxy brightness magnitudes across the different bands (for working on the mixed-input model). This involved some research to see what was the recommended magnitude measurements for galaxies from the SDSS. I decided to go with the cModel fluxes, which are discussed here: https://www.sdss.org/dr17/algorithms/magnitudes/#cmodel.

#### Downloading Code:  <a name="download_code"></a>
Since a lot of this is wrapped up in Luigi tasks to define what the "This step in the DAG successfully ran" output file is, I'll paraphrase my code to the important computations. These two classes handle opening the (already downloaded earlier) tabular data file (csv with magnitudes, object ids, etc), formatting the template url for where the relevant FITS images are stored in the five available filter bands, then downloading them.

Note that ImageDownloader is designed to run in parallel with multiple instances of itself responsible for downloading a smaller range of files than the 50,000 to do in total.

In [ ]:
class URLgenerator(luigi.Task):
    """Luigi task to generate the URLS from the tabular data file"""
    ...
    def run(self):
        """Generates the URLs"""
        df = pd.read_csv(self.input().path)
        urls = []
        bands = ["u", "g", "r", "i", "z"]
        for i, row in df.iterrows():
            for band in bands:
                urls.append(
                    f"https://data.sdss.org/sas/dr17/eboss/photoObj/frames/{row.rerun}/{row.run}/{row.camcol}/frame-{band}-{str(row.run).zfill(6)}-{row.camcol}-{str(row.field).zfill(4)}.fits.bz2\n"
                )
        print("Success: Generated URLs")

        with self.output().open(mode="w") as f:
            for u in urls:
                f.write(u)

class ImageDownloader(luigi.Task):
    """
    Downloads all files in specified range of the urls.txt. Usually used with copies of itself
    with different parameters to span the whole range of urls to download.

    Example::
        n_workers = 10  # should be limited by your hardware
        n_urls = 50000
        chunk = n_urls // n_workers

        luigi.build(
            [data_downloader.Downloader(lower=i, upper=i + chunk) for i in range(0, n_urls, chunk)],
            local_scheduler=True,
            workers=n_workers)
    """
    ...
    
    @classmethod
    @contextmanager
    def download(cls, url) -> ContextManager:
        """Context manager for downloading FITS images from the SDSS

        :param url: string url to fits file on SDSS server
        :return: ContextManager"""

        # Generating temporary and final file names
        tmp = f"{hash(url)}.fits.bz2"
        # skipping the new line character at the end
        file_name = url[-31:-1]

        try:
            # If file already exists, don't download
            if os.path.exists(join("data", "images", f"{file_name}")):
                print("Skipping!!", file_name)
                yield file_name

            else:
                # download file to the tmp file name
                os.system(f"wget -O data/images/{tmp} {url}")
                # atomically move/rename the successful download to the final name
                os.system(f"mv data/images/{tmp} data/images/{file_name}")
                yield file_name
        finally:

            # Cleanup, if tmp still exists
            if os.path.exists(tmp):
                os.remove(tmp)

            if not os.path.exists(join("data", "images", f"{file_name}")):
                # didn't download, try again
                print("Trying again")
                cls.download(url)

    def run(self):
        """Downloads and moves the images from the SDSS atomically"""
        with self.input().open("r") as urls:
            for i, url in enumerate(urls):
                # only downloads the section of urls that have been passed in
                # as Luigi parameter
                if i >= self.lower and i <= self.upper:
                    # using our context manager to download the urls
                    with self.download(url) as d:
                        print(f"Success: {d}")


## Preprocessing <a name="preprocessing"></a>

After downloading the five images (called FITS images, the file format used in astronomy) for each observation, there was about 80GB of compressed (180GB uncompressed) raw data.
<img src='docs/images/preprocess_vis.png' width="400" height="400"> 

The preprocessing looked like above: reading in the zipped file with the built-n bz2 library, opening the image with astropy’s file handing functions, then determining the pixel grid and sky coordinates from this image using astropy’s WCS module, which uses the metadata in every FITS image to determine exactly what part of the sky each pixel in the image corresponds to.

That makes it straightforward to convert from knowing where something is in the sky to where it is in your image. I used that process to center a (32, 32) square on the object in each of the five image frames & crop.

There were a number of challenges I ran into in this step:

- Despite the safeguards in ImageDownloader, empty files still get saved to disk occasionally, so I would get an OSError when starting the preprocessing step.

- There were a few observations in the tabular data that were not downloaded. Either the SDSS didn’t actually have the image corresponding to the tabular data entry or it was stored on a different part of their database. Since this was only a handful of files, I elected to skip them and build in error handling for FileNotFound.

I wrapped the preprocessing in a try except for the two errors I expected to occur and log the files that cause those errors.


### Preprocessing Code  <a name="preprocessing_code"></a>

In [ ]:
# These are the three functions that do all the heavy lifting of the pre-processing

def get_data_cube(observation: pd.Series) -> np.ndarray:
    """
    Takes in a row of the tabular data corresponding to one observation and returns (5, dim, dim) 
    datacube of observations
    """
    dim = int(omegaconf.OmegaConf.load(os.path.join(abs_path, "conf", "preprocessing.yaml"))["dim"])

    bands = ["u", "g", "r", "i", "z"]
    files = []
    for b in bands:
        files.append(
            f"frame-{b}-{str(observation.run).zfill(6)}-{observation.camcol}-{str(observation.field).zfill(4)}.fits.bz2"
        )

    data_cube = np.zeros((5, dim, dim), dtype=np.float64)
    for i, f in enumerate(files):
        data_cube[i] = get_cropped(observation, f, dim)

    # datacube should be (5, dim, dim)
    return data_cube


def get_cropped(observation: pd.Series, fits_file: str, dim: int) -> np.array:
    """
    Given a file name, returns the cropped image as np array centered on the observation
    Unzips the file in memory, converts the RA & DEC of obj to pixel coords, centers on object & crops to 32x32.

    :param observation: row in tabular data corresponding to observation
    :param fits: path to corresponding unzipped fits image
    :param dim: int for one side of square image to return (default is 32)
    :return: np.array of one cropped image to load into datacube
    """
    # config path loading
    local_paths = omegaconf.OmegaConf.load(os.path.join(abs_path, "conf", "local_paths.yaml"))
    fits_file_path = os.path.join(local_paths["data"], local_paths["images"], fits_file)

    # reading the compressed file without extracting it
    with bz2.BZ2File(fits_file_path, "rb") as fits:
        # opening the image from the uncompressed read
        with astropy.io.fits.open(fits) as hdulist:

            # header contains meta data about the image
            header = hdulist[0].header
            # create world coordinate system from header
            wcs = astropy.wcs.WCS(header)
            # get a coordinate object of observation
            coord = astropy.coordinates.SkyCoord(ra=observation.ra, dec=observation.dec, unit="deg")
            # pixel coordinates of center of observation
            target = astropy.wcs.utils.skycoord_to_pixel(coord, wcs)
            target = (int(target[0]), int(target[1]))
            # actual image
            data = hdulist[0].data
            assert type(dim) == int  # sanity check
            # crop the image into (dim, dim) square
            cropped = get_square(data, target, dim)
    return cropped


def get_square(data: np.array, center: tuple, dim: int) -> np.array:
    """
    Crops the FITS image to a (dim, dim) shaped numpy array, centered on
    the object of interest.

    For more info on the WCS for astropy & how it handles (x, y) math vs array
    convention: https://docs.astropy.org/en/stable/wcs/index.html
    https://docs.astropy.org/en/stable/api/astropy.wcs.utils.skycoord_to_pixel.html

    :param data: np.array of the actual image itself
    :param center: (columns, rows) - (x, y) in math perspective
    :param dim: int for square dimensions to return

    :return: np.array of shape: (dim, dim). Is the cropped image.
    """
    center = list(center)

    # Center[0] is column wise, so shape[1]
    if center[0] - dim // 2 < 0:
        center[0] = dim // 2
    if center[0] + dim // 2 > data.shape[1]:
        center[0] = data.shape[0]

    # Center[1] is row wise, so shape[0]
    if center[1] - dim // 2 < 0:
        center[1] = dim // 2
    if center[1] + dim // 2 > data.shape[0]:
        center[1] = data.shape[0]

    return data[center[1] - dim // 2 : center[1] + dim // 2, center[0] - dim // 2 : center[0] + dim // 2].copy()



## Data EDA <a name="eda"></a>
I ended up with 80GB *compressed* raw data (240GB uncompressed). I had to be careful to iterate over the raw data and ensure that there were no unzipped artifacts laying around that would potentially crash my hard drive. 

After the data pre-processing, I ended up with just under 7,800 training points (32x32x5 data cubes) which was only 40MB! The data generator step ultimately wasn't needed at this smaller dataset size, but I still had implemented and tested it, so I continued to use it since it let me also do multiprocessing for the training. 

<img src='docs/images/zdist.png' width="500" height="400">

This is the distribution of redshifts (often just labelled "z", which is confusing because there is also a z-band for photometry as well) for the dataset, which matches up pretty well with the general distribution of SDSS redshifts in general.

### Preprocessed Data: Visualized <a name="eda_vis"></a>
An example pre-processed image! Note that the composite image is not part of the data pipeline, it's just to get a better feel of the relative intensities in the image across the bands. Also, it's only a three channel image, even though our data would be "five" channels.

Blue-Green-Red Composite   |  U-band
:-------------------------:|:-------------------------:
![](docs/images/composite_ugr.png)  |  ![](docs/images/preprocess1.png)

G-band                     |  R-band
:-------------------------:|:-------------------------:
![](docs/images/preprocess2.png)  |  ![](docs/images/preprocess3.png)

I-band                     |  Z-band
:-------------------------:|:-------------------------:
![](docs/images/preprocess4.png)  |  ![](docs/images/preprocess5.png)

We can see that it did successfully crop on each object!

### EDA Stats Visualized <a name="eda_stats"></a>
Calculated from the visualized statistical datacubes for reference:
<img src='docs/images/stats.png' width="400" height="300">

Mean Images across bands   |  .
:-------------------------:|:-------------------------:
![](docs/images/mean_u.png)  |  ![](docs/images/mean_u.png)
![](docs/images/mean_g.png)  |  ![](docs/images/mean_r.png)
![](docs/images/mean_i.png)  |  ![](docs/images/mean_z.png)

The mean images were helpful to sanity check that the preprocessing aligning went smoothly.

Std Images across bands    |  .
:-------------------------:|:-------------------------:
![](docs/images/std_u.png)  |  ![](docs/images/std_u.png)
![](docs/images/std_g.png)  |  ![](docs/images/std_r.png)
![](docs/images/std_i.png)  |  ![](docs/images/std_z.png)

An interesting thing about the std are the very bright blobs on the fringes of the image. I guessed that this would be due to some images having bright other objects in the frame. I checked this by looking at the maximum images across the different bands and found an almost identical max image across all five bands:

<img src='docs/images/max_i.png' width="400" height="300">

Notice how the center of the image is comparatively dim. I think that the blobs that match up perfectly with the std deviation images were so saturated that they skewed the entire dataset for those observations. Presumably there are n-blobs worth of images with saturated extra objects that contributed to this max pixel image.

# Architecture <a name="architecture"></a>

The base for this was the Pasquet et al paper [2], whose associated code repository can be found here: https://github.com/umesh-timalsina/redshift/.
They don't explicitly state what their loss function was in their paper or their optimizer or actually what their smaller architecture was, just that they did work on one for 10k training points. 

Their Architecture         |  My Final Model
:-------------------------:|:-------------------------:
![](docs/images/model.png) |  ![](docs/images/flipmodel.png)

#### Differences: 
Most notably, I added normalization and data-augmentation pre-processing layers and have one less dense layer (with potentially problematic ReLu activation) and two fewer inception module layers. They were also using a parametrized ReLu activation, which I switched to a LeakyReLu when I suspected that part of the extremely fast convergence might be due to a dying ReLU (I suspect) issue.

While the random flip layer doesn't generate any copies of the training data as "extra" training points with flips as regular data augmentation, it does effectively allow you to pretend that you do have more data. With each epoch, instead of seeing every single data point, you can only see 1/4 of all the possible images in the training set, as each image could be randomly flipped horizonally, vertically, or both. 

Since we employ early stopping (100 epochs, but always converges in under 50), it does let the model go over the data as much as it wants.

#### Why Average Pooling?
Also to note: Pasquet says in the paper that they use average pooling instead of max because of the amount of noise from the sky background in most images.

It's very common in astronomy to deal with cosmic rays and other bright sources of noise that will saturate the camera. Usually during the data cleaning and noise subtraction steps done in the catalog data pipelines, you use the median image across observations for the night to help remove these saturating events.

In [53]:
# The final architecture used:
model.model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 5)]  0           []                               
                                                                                                  
 normalization_3 (Normalization  (None, 32, 32, 5)   0           ['input_4[0][0]']                
 )                                                                                                
                                                                                                  
 random_flip_3 (RandomFlip)     (None, 32, 32, 5)    0           ['normalization_3[0][0]']        
                                                                                                  
 conv2d_51 (Conv2D)             (None, 32, 32, 64)   8064        ['random_flip_3[0][0]']    

### Architecture Code <a name="arch_code"></a>

In [ ]:
@dataclass
class BaseModel:
    """
    Base class for Model for this pipeline. Uses lazy-loading for most heavy-lifting attributes

    :param batch_size: int - specified via config file
    :param test_split: float - specified via config file
    :param num_classes: int - specified via config file
    :param lr: float - specified via config file
    :param seed: int - specified via config file
    :param input_img_shape: tuple - specified via config file
    """

    batch_size: int
    test_split: float
    num_classes: int
    seed: int
    lr: float
    input_img_shape: tuple
    epochs: int

    @cached_property
    def labels(self) -> dict:
        """
        Creates dict of series object, with objID as the index for easy partition access to
        labels.
        :return: dict partitioned by 'test' and 'train' of pandas Series with labels
        """
        tabular_name = OmegaConf.load(join(abs_path, "conf", "aws_paths.yaml"))["tabular_data"]
        tabular_path = join(abs_path, "..", "data", tabular_name)
        df = pd.read_csv(tabular_path, usecols=["objID", "z"])

        series = pd.Series(data=df.z.values, index=df.objID, name="z")

        return {"test": series.loc[self.partition["test"]], "train": series.loc[self.partition["train"]]}

    @cached_property
    def partition(self) -> dict:
        """
        Creates the partition for testing & training by object ID.

        This is used by labels to split the labels into test & train sets and also by DataGenerator to
        create batches, etc.

        :return: dict of string file names partitioned by 'test' and 'train'
        """
        # get a numpy array of all the processed data ready for training
        dataset = glob.glob(join("data", "processed", "*.npy"))
        func = lambda file: int(file[15:-15])
        dataset = np.array(list(map(func, dataset)))

        # this is a numpy array of integers, which are the object ID's. Each processed datacube
        # is named with its object ID
        unique_data = np.unique(dataset)

        # random shuffle
        np.random.shuffle(unique_data)

        partition = {}

        # the test/train split, note the hard to spot colons for slicing
        partition["test"] = unique_data[: int(len(unique_data) * self.test_split)]
        partition["train"] = unique_data[int(len(unique_data) * self.test_split) :]

        return partition

    @cached_property
    def training_generator(self) -> DataGenerator:
        """
        Cached property, so only gets called once when the property is accessed. Part of BaseModel class.
        Creates an intance of the DataGenerator class with the relevant test/train partition to use for training
        or prediction.
        """
        return DataGenerator(self.partition["train"], self.labels["train"], self.num_classes, batch_size=self.batch_size)  # , **params)

    @cached_property
    def testing_generator(self) -> DataGenerator:
        """
        Cached property, so only gets called once when the property is accessed. Part of BaseModel class.
        Creates an intance of the DataGenerator class with the relevant test/train partition to use for training
        or prediction.
        """
        return DataGenerator(self.partition["test"], self.labels["test"], self.num_classes, batch_size=self.batch_size)  # , **params)

    def train(self, checkpoint_path, history_path, workers=12):
        """
        Wrapper for fitting the model with the generator we created
        Note we can only use multiprocessing because we're passing in a generator

        :param checkpoint_path: str
        :param history_path: str
        :param workers: int
        """
        my_callbacks = [
            tf.keras.callbacks.CSVLogger(history_path, separator=",", append=True),
            tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True),
            tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.0001, patience=3, verbose=1, mode="auto"),
        ]

        history = self.model.fit(
            self.training_generator,
            validation_data=self.testing_generator,
            epochs=epochs,
            use_multiprocessing=True,
            callbacks=my_callbacks,
            workers=workers
        )
        return history

    def loss(self, true, pred):
        """
        Custom loss function for keras
        True should be a float, pred should be a (num_classes,) shaped array.
        Calculates the MSE between the true value and the expected value of the output
        """
        return tf.square(tf.subtract(true, tf.reduce_sum(tf.multiply(self.bin_values, pred))))

    @cached_property
    def bin_values(self):
        """returns array with the same length as the number of classes, with the midpoint of the start and end range
        stored (ie, the bin containing redshift values 0 to 0.2 will have bin value 0.1)
        """
        max_val = 0.4  # since 0.39.. is the max redshift value that we see
        bins = np.linspace(0, max_val, self.num_classes, endpoint=False, dtype=np.float32)
        bin_values = bins + bins[1] / 2
        return bin_values


class InceptionModel(BaseModel):
    """
    Inception model from Pasquet et al paper. Wrapper for many tensorflow/keras functions.

    :param batch_size: int - specified via config file
    :param test_split: float - specified via config file
    :param num_classes: int - specified via config file
    :param lr: float - specified via config file
    :param seed: int - specified via config file
    :param input_img_shape: tuple - specified via config file
    """
    @cached_property
    def model(self):
        """
        Constructs the model architecture. Most of the keras code here is all from the Inception model
        in https://github.com/umesh-timalsina/redshift/blob/3c11608d5818ae58ab2ce084832b56766858b3a1/model/model.py.
        Which is the code repository for the Pasquet et al 2018 paper
        """
        # Input Layer Galactic Images
        image_input = Input(shape=self.input_img_shape[1:])

        # Adding a preprocessing layer for normalization
        norm = Normalization(mean=np.load('data/stats/mean.npy'), variance=np.load("data/stats/variance.npy"), axis=(1,2,3))
        norm_out = norm(image_input)

        # adding a random flip
        flip = RandomFlip()
        flip_out = flip(norm_out)

        # Back to their code
        # Convolution Layer 1
        conv_1 = Conv2D(64, kernel_size=(5, 5), padding="same", activation=LeakyReLU())
        conv_1_out = conv_1(flip_out)

        # Pooling Layer 1
        pooling_layer1 = AveragePooling2D(pool_size=(2, 2), strides=2, padding="same")
        pooling_layer1_out = pooling_layer1(conv_1_out)

        # Inception Layer 1
        inception_layer1_out = self.add_inception_layer(pooling_layer1_out, num_f1=48, num_f2=64)

        # Pooling Layer 2
        pooling_layer2 = AveragePooling2D(pool_size=(2, 2), strides=2, padding="same")
        pooling_layer2_out = pooling_layer2(inception_layer1_out) # changed from 2

        # Inception Layer 3
        inception_layer3_out = self.add_inception_layer(pooling_layer2_out, 92, 128)

        # Pooling Layer 3
        pooling_layer3 = AveragePooling2D(pool_size=(2, 2), strides=2, padding="same")
        pooling_layer3_out = pooling_layer3(inception_layer3_out)  # modified 4 -> 3

        # Inception Layer 5
        inception_layer5_out = self.add_inception_layer(pooling_layer3_out, 92, 128, kernel_5=False)

        # input_to_pooling = cur_inception_in
        input_to_dense = Flatten(data_format="channels_last")(inception_layer5_out)
        model_output = Dense(units=self.num_classes, activation="softmax")(input_to_dense)

        ### Construction, summary, and compiling
        model = tensorflow.keras.Model(inputs=[image_input], outputs=model_output)

        model.summary()
        opt = Adam(learning_rate=self.lr, epsilon=self.epsilon)
        model.compile(optimizer=opt, loss=self.loss)
        return model

    def add_inception_layer(self, input_weights, num_f1, num_f2, kernel_5=True):
        """
        These convolutional layers take care of the inception layer.
        This function is also not my code, also directly from the Pasquet et al paper repo.
        """
        # Conv Layer 1 and Layer 2: Feed them to convolution layers 5 and 6
        c1 = Conv2D(num_f1, kernel_size=(1, 1), padding="same", activation=LeakyReLU())
        c1_out = c1(input_weights)
        if kernel_5:
            c2 = Conv2D(num_f1, kernel_size=(1, 1), padding="same", activation=LeakyReLU())
            c2_out = c2(input_weights)

        # Conv Layer 3 : Feed to pooling layer 1
        c3 = Conv2D(num_f1, kernel_size=(1, 1), padding="same", activation=LeakyReLU())
        c3_out = c3(input_weights)

        # Conv Layer 4: Feed directly to concat
        c4 = Conv2D(num_f2, kernel_size=(1, 1), padding="same", activation=LeakyReLU())
        c4_out = c4(input_weights)

        # Conv Layer 5: Feed from c1, feed to concat
        c5 = Conv2D(num_f2, kernel_size=(3, 3), padding="same", activation=LeakyReLU())
        c5_out = c5(c1_out)

        # Conv Layer 6: Feed from c2, feed to concat
        if kernel_5:
            c6 = Conv2D(num_f2, kernel_size=(5, 5), padding="same", activation=LeakyReLU())
            c6_out = c6(c2_out)

        # Pooling Layer 1: Feed from conv3, feed to concat
        p1 = AveragePooling2D(pool_size=(2, 2), strides=1, padding="same")
        p1_out = p1(c3_out)

        if kernel_5:
            return concatenate([c4_out, c5_out, c6_out, p1_out])
        else:
            return concatenate([c4_out, c5_out, p1_out])


### Training Example <a name="train_code"></a>

In [54]:
# Model Training example, bypassing the Luigi Task for this:
from final_project import models
import datetime

time = datetime.datetime.now()
prefix = "demo"+str(time.hour)+str(time.minute)
num_classes = 100
model = models.InceptionModel(batch_size=32, test_split=0.2, num_classes=num_classes, epochs=50,
                              lr=0.001, seed=1, input_img_shape=(32, 32, 32,5))

history = model.train(f"data/models/{prefix}_checkpoint", f'data/models/{prefix}_history.csv')
model.model.save(f"data/models/{prefix}_epoch")
print(model.model.evaluate(model.testing_generator))

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 5)]  0           []                               
                                                                                                  
 normalization_4 (Normalization  (None, 32, 32, 5)   0           ['input_5[0][0]']                
 )                                                                                                
                                                                                                  
 random_flip_4 (RandomFlip)     (None, 32, 32, 5)    0           ['normalization_4[0][0]']        
                                                                                                  
 conv2d_68 (Conv2D)             (None, 32, 32, 64)   8064        ['random_flip_4[0][0]']    

Leftover:  20
Leftover:  13
Epoch 1/50
193/193 [==============================] - ETA: 0s - loss: 22.0817
Epoch 1: val_loss improved from inf to 0.19475, saving model to data/models/demo2345_checkpoint


INFO:tensorflow:Assets written to: data/models/demo2345_checkpoint/assets


INFO:tensorflow:Assets written to: data/models/demo2345_checkpoint/assets


193/193 [==============================] - 22s 105ms/step - loss: 22.0817 - val_loss: 0.1948
Epoch 2/50
193/193 [==============================] - ETA: 0s - loss: 0.0370
Epoch 2: val_loss improved from 0.19475 to 0.01936, saving model to data/models/demo2345_checkpoint


...

193/193 [==============================] - 24s 121ms/step - loss: 0.0101 - val_loss: 0.0100
Epoch 16/50
193/193 [==============================] - ETA: 0s - loss: 0.0101
Epoch 16: val_loss improved from 0.00996 to 0.00991, saving model to data/models/demo2345_checkpoint


INFO:tensorflow:Assets written to: data/models/demo2345_checkpoint/assets


INFO:tensorflow:Assets written to: data/models/demo2345_checkpoint/assets


193/193 [==============================] - 25s 123ms/step - loss: 0.0101 - val_loss: 0.0099
Epoch 17/50
193/193 [==============================] - ETA: 0s - loss: 0.0101

KeyboardInterrupt: 

### Data Generator
Thank you so much for your suggestion of that Stanford blog!! [3] This worked out so well. Even though my data ended up fitting in memory (~40MB, down from the 80GB raw), this still allowed for multiprocessing, as long as I kept the data generator thread-safe. 

In [ ]:

class DataGenerator(tf.keras.utils.Sequence):
    """
    Generator for Keras training to allow multiprocessing and training on batches with only the
    batch itself being loaded into memory.

    This implementation was heavily inspired by various examples for creating data generators
    for keras, namely https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.
    """

    def __init__(
        self,
        data_IDs: np.array,
        labels: pd.Series,
        n_classes: int,
        batch_size: int = 64,
        dim: tuple = (32, 32, 5),
    ):
        self.data_IDs = data_IDs
        self.labels = labels
        self.n_classes = n_classes
        self.dim = dim
        self.batch_size = batch_size
        self.batch = self.get_batch()
        self.indexes = self.get_indices()
        self.shuffle = True

    def get_batch(self):
        """
        Generating upon initalization, so we can keep things thread-safe for multiprocessing.
        This is a little messy, but I've tested the function pretty well
        """
        batch = {}
        num_batches = len(self.data_IDs) // self.batch_size
        print("Leftover: ", len(self.data_IDs) % self.batch_size)
        j = 1
        for i in range(num_batches):
            batch[i] = self.data_IDs[i * self.batch_size : j * self.batch_size]
            j += 1

        return batch

    def get_indices(self):
        """
        Returns indices for __getitem__. Want there to be the same number of indices as there are batches
        to run.
        """
        return np.arange(len(self.data_IDs) // self.batch_size)

    def __len__(self):
        """Number of batches per epoch"""
        return len(self.data_IDs) // self.batch_size

    def __getitem__(self, index):
        """Generate one batch of data"""
        X, y = self._data_generation(self.batch[index])

        return X, y

    def on_epoch_end(self):
        """Updates indexes with shuffle after each epoch"""
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _data_generation(self, batch):
        """
        From file names for a batch, generate the data.
        """
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(batch):
            x = glob.glob("data/processed/" + str(ID) + "-*.npy")[0]
            # I saved the datacubes as (5, 32, 32) instead of (32, 32, 5) as they need to be for training
            X[i] = np.transpose(np.load(os.path.join(abs_path, "..", x)), (1, 2, 0))

            # ID is not entirely unique in the csv, so occasionally I get one objID with two spectroscopic redshifts
            # I decided to handle this by averaging the estimates (it's always two)
            if type(self.labels[ID]) == pd.Series:
                y[i] = np.mean(self.labels[ID].values)
            else:
                y[i] = self.labels[ID]

        return X, y

## Steps to Improve Model <a name="improving"></a>
Out of the box, the original architecture from Pasquet found at this [repo](https://github.com/umesh-timalsina/redshift/) did not work for me at all (you have to look at a few commits back to see their model before they transitioned the model to something called DeepForge).

**The original paper's code used a sparse categorical cross entropy loss**, which neglects what our "categories" really mean - since an estimate that's in a nearby bin is better than an estimate that's in a far away bin. So the model is still outputing a rough PDF of redshift distribution, but the new loss function evaluates the MSE of the expected value of the redshift prediction as detailed here  [5]:
<img src='docs/images/expectedvalue.png' width="700" height="400">


In [ ]:
# This was the loss function I wrote for training instead of the sparse_categorical_crossentropy that the 
# paper repo was using.
    def loss(self, true, pred):
        """
        Custom loss function for keras
        True should be a float, pred should be a (num_classes,) shaped array.
        Calculates the MSE between the true value and the expected value of the output
        """
        return tf.square(tf.subtract(true, tf.reduce_sum(tf.multiply(self.bin_values, pred))))


It was initially very discouraging results with the model, even with the modified loss function. I experienced results that often looked like this with completely flatlining training, even with the full architecture:
<img src='docs/images/extraining.png' width="400" height="400">

(Keep in mind that my redshift values range from 0 to 0.4.)

The model would converge to often a terrible loss after only one epoch and I would never see the validation loss change. It would early stop after my patience=5 had been reached. I thought it might be a problem with the PRelu activations from the original paper dying off, especially as my loss is initially above 100 and drops dramatically in the first epoch. The validation loss would also always be just slightly below the training loss, which gave me pause. After rulling data leaks and other things out, I think that it's because the validation loss is being evaluated *after* the training for that epoch.

I switched to LeakyRelu to avoid the vanishing gradients, but didn't see any change in behavior. I combed through my code to make sure that there weren't any errors, but couldn't find anything. 

#### Data Augmentation & Normalization:
Experiments to tried:
- smaller architecture
    - One dense layer (instead of two)
    - Three inception modules (instead of five)
- smaller with normalized images
    - computed statistics from entire dataset and passed into keras Normalization layer
- smaller with normalized images and data augmentation
    - used Keras RandomFlip layer with horizontal and vertical flipping
- different hyperparameters: 
    - batch size: 32, 64
    - learning rates: 0.0001, 0.0005, 0.001, 0.01, 1.0, and 10.0 
    - epsilon: 1e-7 (default), 0.1, 1.0
- different activation functions: 
    - PReLu
    - LeakyReLu
    - TanH (didn't work)
    
It consistently converged after only one epoch. Used early stopping for most of it (with a high patience of 5, saving best model checkpoint). Validation loss was usually smaller than training loss after the first epoch (loss would start at 100ish and drop to 1-6 range usually).

I struggled with the convergence of the model after only one epoch with every variation - LR, normalization, batch size, leaky RELU, parametrized RELU, etc. Then I read that the InceptionV3 authors suggested using a very large epilson value for ADAM (1.0 or 0.1 instead of 1e-7) [1].

Luckily, that was the breakthrough! While the normalization and data augmentation further improved things, the model went from converging in one epoch, often to very large losses (often above ten), to actually changing across epochs.

## Results
Overall, there were two models that stood out. Both had the same, smaller architecture, Adam lr of 0.0001, Adam epsilon of 1.0, and loss function, except the the RandomFlip keras data augmentation layer.

**Below is a summary of a few models (and the best):**
<img src='docs/images/results.png' width="600" height="600">

While they both achieved very similar results, I think that the data augmented model will generalize better, as it had to learn to accomodate horizontal and vertical flips. It makes sense that the model should be agnostic to whether or not there was an extra mirror in the telescope or if the camera was mounted upside down.

<img src='docs/images/predpdf.png' width="400" height="400"> 

Above is a visualized PDF of a prediction from the augmented model (it had predicted almost zero for the rest of the bins to the right, so the plot is "zoomed in").

**Data Augmented Model Training History:**
<img src='docs/images/most_acc_dataaug.png' width="400" height="400"> 

## Concerns
The data is not very balanced. As you can see in the EDA section where we look at a histogram of the redshift values, there are very few in the dataset above z=0.3. Most of the data would be contained in eight bins (for the width I used) centered around z=0.1 in the final prediction layer. I would also like to evaluate the model with different metrics like Probability Integral Transform (PIT) and the Continuous Ranked Probability Score (CRPS) to examine how the model has really learned the distribution.  
I'm also not sure if I incentivized the right behavior by softmaxing and then the MSE of the expected value loss that took this to a regression problem. The redshift problem is known to be a multimodal problem [6] and this loss function neglects that. In the future, I would like to find a loss that respects the nature of the problem more. I would also like to visualize the kernels being learned in the model to better understand what the model is really learning.

## Sources <a name="sources"></a>
[1] InceptionV3 paper with large epilson https://arxiv.org/abs/1512.00567  
[2] Pasquet paper https://arxiv.org/abs/1806.06607  
[3] Stanford data generator for keras https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly) 

[4] Pasquet repository with architecture https://github.com/umesh-timalsina/redshift/  
[5] Expected Value of bins blurb https://www.aanda.org/articles/aa/full_html/2019/01/aa33617-18/aa33617-18.html  
[6] Multimodal https://ui.adsabs.harvard.edu/abs/2016arXiv160606094K/abstract